In [ ]:
'''
text data 학습을 위한 코드
- input : df_all_txt.csv (preprocessing.py 실행 후 생성됨.)
- output : text_train_model.pt (text data 학습 모델)
'''

'\ntext data 학습을 위한 코드\n- input : df_all_txt.csv (preprocessing.py 실행 후 생성됨.)\n- output : text_train_model.pt (text data 학습 모델)\n'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install --upgrade numpy==1.23.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xgboost 2.1.1 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.
albucore 0.0.13 requires numpy<2,>=1.24.4, but you have numpy 1.23.1 which is incompatible.
albumentations 1.4.13 requires numpy>=1.24.4, but you have numpy 1.23.1 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.1 which is incompatible.
pandas-stubs 2.1.4.231227 requires numpy>=1.26.0; python_version < "3.13", but you have numpy 1.23.1 which is incompatible.
tensorflow 2.17.0 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11

In [1]:
!pip install mxnet
!pip install gluonnlp tqdm
!pip install sentencepiece
!pip install transformers

  Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached graphviz-0.8.4-py2.py3-none-any.whl.metadata (6.4 kB)
Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl (49.1 MB)
Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661659 sha256=6a073b80372eb5a9b50d792c6ce19b5a02334e54fd337d4e25eb3947f45153a1
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-pbl4vfyr/kobert-tokenizer_268be021cf5c4103991fd79ba89e2945
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-pbl4vfyr/kobert-tokenizer_268be021cf5c4103991fd79ba89e2945
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=0d66e6818a0ef4a3d8ef7b9910030d903e6a8e68c652e5757d3b05613243b8cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-bc5klb6u/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-t2npyvo2
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-t2npy

In [3]:
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from torch import nn
from torch.utils.data import Dataset

import torch
import numpy as np
import pandas as pd
import gluonnlp as nlp

In [4]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [6]:
# GPU 사용 시
device = torch.device("cuda:0")

In [7]:
# 데이터셋 전처리, df_all_txt.csv 파일 올바른 디렉토리에 위치
# data split -> 15499(0.8)+3875(0.2)

df_all_txt = pd.read_csv("/content/drive/MyDrive/ETRI_Text_Audio_Multimodal-main/data/df_all_txt.csv")
df_all_txt = df_all_txt[:15499] # train data

In [9]:
df_all_txt.loc[(df_all_txt['Label'] == "neutral"), 'Label'] = 0
df_all_txt.loc[(df_all_txt['Label'] == "happiness"), 'Label'] = 1
df_all_txt.loc[(df_all_txt['Label'] == "surprise"), 'Label'] = 2
df_all_txt.loc[(df_all_txt['Label'] == "sadness"), 'Label'] = 3
df_all_txt.loc[(df_all_txt['Label'] == "angry"), 'Label'] = 4
df_all_txt.loc[(df_all_txt['Label'] == "disgust"), 'Label'] = 5
df_all_txt.loc[(df_all_txt['Label'] == "fear"), 'Label'] = 6

In [10]:
train_data_list = []
for ques, label in zip(df_all_txt['text'], df_all_txt['Label'])  :
    train_data = []
    train_data.append(ques)
    train_data.append(str(label))
    train_data_list.append(train_data)

In [11]:
text_all_emotion = []
for i in range(len(train_data_list)):
    text_all_emotion.append(train_data_list[i][1])

In [12]:
# class BERTDataset(Dataset):
#     def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
#         transform = nlp.data.BERTSentenceTransform(
#             bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

#         self.sentences = [transform([i[sent_idx]]) for i in dataset]
#         self.labels = [np.int32(i[label_idx]) for i in dataset]

#     def __getitem__(self, i):
#         return (self.sentences[i] + (self.labels[i], ))

#     def __len__(self):
#         return (len(self.labels))

In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        self.sentences = [(np.array(tokenizer.encode(i[sent_idx], max_length=512, pad_to_max_length=True)).astype(np.int32),
                           np.array(len(tokenizer.encode(i[sent_idx], max_length=512, pad_to_max_length=False))).astype(np.int32),
                           np.zeros(shape=(64,), dtype=np.int32)) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

        print(self.sentences[0])

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [14]:
# 파라미터 세팅
max_len = 128
batch_size = 16
warmup_ratio = 0.1
num_epochs = 16
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [15]:
tok = tokenizer.tokenize
data_train = BERTDataset(train_data_list, 0, 1, tok, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(array([   2,  517, 6896, 7767, 7581, 7096, 7199, 4102, 4207, 7848,   54,
       1185,  907, 1100, 7086, 2705, 7119,  905,  830, 6834,   54,    3,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,  

In [16]:
# class BERTClassifier(nn.Module):
#     def __init__(self,
#                  bert,
#                  hidden_size=768,
#                  num_classes=7,
#                  dr_rate=None,
#                  params=None):
#         super(BERTClassifier, self).__init__()
#         self.bert = bert
#         self.dr_rate = dr_rate

#         self.classifier = nn.Linear(hidden_size , num_classes)
#         if dr_rate:
#             self.dropout = nn.Dropout(p=dr_rate)

#     def gen_attention_mask(self, token_ids, valid_length):
#         attention_mask = torch.zeros_like(token_ids)
#         for i, v in enumerate(valid_length):
#             attention_mask[i][:v] = 1
#         return attention_mask.float()

#     def forward(self, token_ids, valid_length, segment_ids):
#         attention_mask = self.gen_attention_mask(token_ids, valid_length)

#         _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
#         if self.dr_rate:
#             out = self.dropout(pooler)
#         return self.classifier(out)

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes = 7, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        segment_ids = segment_ids.repeat(1, token_ids.shape[1] // segment_ids.shape[1])

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [19]:
# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [20]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중 분류를 위한 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_history = []
val_history = []
loss_history = []

In [ ]:
import time

for e in range(num_epochs):
    start_time = time.time()  # 에폭 시작 시간 기록
    train_acc = 0.0
    val_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())

    epoch_time = time.time() - start_time  # 에폭 종료 시간에서 시작 시간을 뺀 결과로 에폭 소요 시간 계산
    print("epoch {} train acc {} epoch time {:.2f} seconds".format(e+1, train_acc / (batch_id+1), epoch_time))


epoch 1 batch id 1 loss 1.8169898986816406 train acc 0.3125
epoch 1 batch id 201 loss 1.935058832168579 train acc 0.23911691542288557
epoch 1 batch id 401 loss 1.2626891136169434 train acc 0.342425187032419
epoch 1 batch id 601 loss 0.8403024077415466 train acc 0.4301164725457571
epoch 1 batch id 801 loss 1.0360132455825806 train acc 0.4751092384519351
epoch 1 train acc 0.47695609344216156 epoch time 1376.87 seconds
epoch 2 batch id 1 loss 1.808354377746582 train acc 0.375
epoch 2 batch id 201 loss 1.1397510766983032 train acc 0.6013681592039801
epoch 2 batch id 401 loss 0.8462741374969482 train acc 0.6022443890274314
epoch 2 batch id 601 loss 0.7034014463424683 train acc 0.6190723793677204
epoch 2 batch id 801 loss 0.8727670311927795 train acc 0.6259363295880149
epoch 2 train acc 0.6055328830096632 epoch time 1376.74 seconds
epoch 3 batch id 1 loss 1.248261570930481 train acc 0.5625
epoch 3 batch id 201 loss 1.011572241783142 train acc 0.6573383084577115
epoch 3 batch id 401 loss 0.56

In [ ]:
import matplotlib.pyplot as plt

# 학습 손실(loss) 시각화
plt.figure(figsize=(12, 6))
plt.plot(loss_history, label='Training Loss')
plt.title('Training Loss over Batches')
plt.xlabel('Batch number')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# 학습 정확도(accuracy) 시각화
plt.figure(figsize=(12, 6))
plt.plot(train_history, label='Training Accuracy')
plt.title('Training Accuracy over Batches')
plt.xlabel('Batch number')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 학습 모델 저장
torch.save(model, "text_train.pt")